# SRF Cavity - Losses Summary

This notebook summarizes main loss channels and their estimated impact on the half-TESLA cavity, used for quantum information processing. The limitations to the total quality factor should be summed over all loss channels, i.e.
$$\dfrac{1}{Q_\text{total}} = \sum _i \dfrac{1}{Q_i}.$$
However, here some loss channels are modeled repeatedly using different methods. The transmon effect on the cavity mode, for example, is modeled by adding resistance to the transmon or by making the sapphire bulk lossy, but these limitations should not be summed together.





In [1]:
import cavity_losses
project = cavity_losses.HFSSproject(project_path = '.',
                             project_name = 'CavityLosses')

## Seam Loss
Here we use the equation $$\dfrac{1}{Q_{\text{seam}}} = \dfrac{L}{G_{\text{seam}}} \left[ 
\dfrac{\int _{\text{seam}} \left| \vec{J}_s \times \hat{l}\right|^{2}dl}{\omega  \int _{\text{total}} \mu_0 \left|\vec{H}\right|^2 dV}
\right]    = \dfrac{y_{\text{seam}} }{g_{\text{seam}}},$$
with the value $g_{\text{seam}}=10^{6} \ 1/\Omega m$ estimated from Brecht's thesis.

In [2]:
project.calculate_seam_loss('SeamLoss', modes = [0,1] , g_seam = 1e6, seam_line = 'SeamLine', volume= 'Vacuum')
print(project.quality_factors)

INFO 10:16AM [connect]: Connecting to Ansys Desktop API...
INFO 10:16AM [load_ansys_project]: 	File path to HFSS project found.
INFO 10:16AM [load_ansys_project]: 	Opened Ansys App
INFO 10:16AM [load_ansys_project]: 	Opened Ansys Desktop v2020.2.0
INFO 10:16AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/barkayg/Dropbox (Weizmann Institute)/Quantum Circuits Lab/Barkay/SRF Cavity/Simulations/LossesCalculations/LossesCalculationsExample/
	Project:   CavityLosses
INFO 10:16AM [connect]: 	Opened active design
	Design:    SeamLoss [Solution type: Eigenmode]
INFO 10:16AM [get_setup]: 	Opened setup `Setup1`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 10:16AM [connect]: 	Connection to Ansys established successfully. 😀 

INFO 10:16AM [analyze]: Analyzing setup Setup1
Design "SeamLoss" info:
	# eigenmodes    7
	# variations    1

Mode 0 results:
y_seam = 1.27e-07 /(Ω*m)
Q_seam = 7.87e+12
tau_seam = 2.87e+02 s

Mode 1 results:
y_seam = 2.21e-02 /(Ω*m)
Q_seam = 4.52e+07
tau_seam 

## Geometry and Filling Factors
Here we use the equation $$ G =\dfrac{\omega \mu_0}{\lambda p_\text{cond}} =  \omega \mu_0 \dfrac{\int _{V} \left|{\vec{H}}\right|^2 dv}{\int _{S} \left|{\vec{H}}\right|^2 ds} \ , \ F= \dfrac{\int _{V_h} \epsilon_h \left|{\vec{E}}\right|^2 dv}{\int _{V} \epsilon_0 \left|{\vec{E}}\right|^2 dv} \approx \dfrac{t_h \int _{S}  \left|{\vec{E}}\right|^2 / (\epsilon_h/\epsilon_0) ds}{\int _{V}  \left|{\vec{E}}\right|^2 dv},$$
with the values $\epsilon_h = 33 \epsilon_0$ (https://journals.aps.org/prl/pdf/10.1103/PhysRevLett.119.264801) and $t_h = 5 \ nm$ dielectric thickness. The nominator integral of the filling factor is divided by the relative premittivity since the electric field ($\vec{E}$) there is perpendicular to the vacuum-metal interface and hence smaller by that factor. See https://phys.libretexts.org/Bookshelves/Electricity_and_Magnetism/Book%3A_Electricity_and_Magnetism_(Tatum)/05%3A_Capacitors/5.14%3A__Mixed_Dielectrics.

We then extrapolate and estimate the quality factor from Romanenko's paper:
$$ Q_\text{TLS} = \dfrac{F_\text{Ref} }{F_\text{Simul}} Q_\text{Ref}$$

In [4]:
project.calculate_GF_factors('GF_Factors', epsilon_r = 33, t_h = 5e-9, volume= 'Vacuum', modes = [0])

print('TESLA cavity has G = 270 Ω') #10.1103/physrevstab.3.092001
print("Romanenko's cavity has approx F = 1.2e-8") #10.1103/PhysRevApplied.13.034032
Q = 1/(5.6e-11 * (project.quality_factors['mode 0']['filling_factor'] / 1.2e-8))
print(f"Estimated quality factor by comparison to Romanenko's cavity is approx Q = {Q:.3}")

INFO 10:17AM [connect]: Connecting to Ansys Desktop API...
INFO 10:17AM [load_ansys_project]: 	File path to HFSS project found.
INFO 10:17AM [load_ansys_project]: 	Opened Ansys App
INFO 10:17AM [load_ansys_project]: 	Opened Ansys Desktop v2020.2.0
INFO 10:17AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/barkayg/Dropbox (Weizmann Institute)/Quantum Circuits Lab/Barkay/SRF Cavity/Simulations/LossesCalculations/LossesCalculationsExample/
	Project:   CavityLosses
INFO 10:17AM [connect]: 	Opened active design
	Design:    GF_Factors [Solution type: Eigenmode]
INFO 10:17AM [get_setup]: 	Opened setup `Setup1`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 10:17AM [connect]: 	Connection to Ansys established successfully. 😀 

INFO 10:17AM [analyze]: Analyzing setup Setup1
Design "GF_Factors" info:
	# eigenmodes    7
	# variations    1

Mode 0 results:
Geometry factor, G = 211.31 Ω
Filling factor, F = 1.3e-08
TESLA cavity has G = 270 Ω
Romanenko's cavity has approx F = 1.2e-8
Esti

## Surface Losses - not calculated in this notebook!
Here we calculate the surface participation ratios in order to estimate the losses due to them. 
The estimation is a very rough one, unlike the more complex, and probably more accurate, method used in http://aip.scitation.org/doi/10.1063/1.4934486. 

Similar to the calculation of the filling factor in the previous subsection we now calculate the participation ratio as
$$    p_i = \dfrac{t \int_{S_i} \left| \vec{E} \cdot ( \vec{E}_\bot /\epsilon_r + \vec{E}_\parallel \times \epsilon_r) \right| ^{2} ds}{\int _{V} \left|\vec{E}\right| ^2 dv},$$
where $i$ is the index of the interface over which the integral is done (MA, MS or SA). We use the values $\epsilon = 10 \epsilon_0$ and $t=3$ nm.

The upper bound to the quality factor is then inversely-proportional to this participation ratio, i.e.
$$ Q_i \leq \dfrac{1}{p_i \tan \delta_i}.$$
For that we use upper bounds $\tan \delta_{MS} = 2.6 \times 10^{-3}, \ \tan \delta_{MA} = 2.1 \times 10^{-2}$ and $\tan \delta_{SA} = 2.2 \times 10^{-3}$. 

## Hybridization with Resistive Transmon
Here we add resistance to the transmon in parallel to the Josephson inductance. This is made in order to model the actual lifetime of a transmon qubit, which is much lower than the $Q \sim 10^{12}$ predicted by the classical HFSS analysis.  

In [5]:
project.get_quality_factor(design_name='ResistiveTransmon', modes = [0,1], print_results=True)

INFO 10:17AM [connect]: Connecting to Ansys Desktop API...
INFO 10:17AM [load_ansys_project]: 	File path to HFSS project found.
INFO 10:17AM [load_ansys_project]: 	Opened Ansys App
INFO 10:17AM [load_ansys_project]: 	Opened Ansys Desktop v2020.2.0
INFO 10:17AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/barkayg/Dropbox (Weizmann Institute)/Quantum Circuits Lab/Barkay/SRF Cavity/Simulations/LossesCalculations/LossesCalculationsExample/
	Project:   CavityLosses
INFO 10:17AM [connect]: 	Opened active design
	Design:    ResistiveTransmon [Solution type: Eigenmode]
INFO 10:17AM [get_setup]: 	Opened setup `Setup1`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 10:17AM [connect]: 	Connection to Ansys established successfully. 😀 

INFO 10:17AM [analyze]: Analyzing setup Setup1
Design "ResistiveTransmon" info:
	# eigenmodes    7
	# variations    1
      Freq. (GHz)  Quality Factor   Lifetime (s)
mode                                            
0        4.371611    1.982010e+10  

## Sapphire Bulk Loss
Here we add loss tangent to the sapphire chip. This is made in order to model the actual lifetime of a transmon qubit, which is much lower than the $Q \sim 10^{12}$ predicted by the classical HFSS analysis.  
Then we compare the effect of this loss tangent on the transmon mode and the cavity mode and calculate the ratio $Q_{cavity}/Q_{transmon}$.

In [6]:
project.get_quality_factor(design_name='SapphireBulkLoss', modes = [0,1], print_results=True)

INFO 10:17AM [connect]: Connecting to Ansys Desktop API...
INFO 10:17AM [load_ansys_project]: 	File path to HFSS project found.
INFO 10:17AM [load_ansys_project]: 	Opened Ansys App
INFO 10:17AM [load_ansys_project]: 	Opened Ansys Desktop v2020.2.0
INFO 10:17AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/barkayg/Dropbox (Weizmann Institute)/Quantum Circuits Lab/Barkay/SRF Cavity/Simulations/LossesCalculations/LossesCalculationsExample/
	Project:   CavityLosses
INFO 10:17AM [connect]: 	Opened active design
	Design:    SapphireBulkLoss [Solution type: Eigenmode]
INFO 10:17AM [get_setup]: 	Opened setup `Setup1`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 10:17AM [connect]: 	Connection to Ansys established successfully. 😀 

INFO 10:17AM [analyze]: Analyzing setup Setup1
Design "SapphireBulkLoss" info:
	# eigenmodes    7
	# variations    1
      Freq. (GHz)  Quality Factor  Lifetime (s)
mode                                           
0        4.371560    2.295939e+10      

## Coupling Losses
Here we add coupling losses. Three coupling pins are present in the device and each one of them causes quality degradations when it is "turned-on" by making it a $50 \Omega$ resistive terminator. We get each coupling quality factor separately and then the quality factor when they are all active. One should verify that
$$Q^{-1}_\text{Ctotal} \approx Q^{-1}_\text{Creadout} + Q^{-1}_\text{Cdrive} + Q^{-1}_\text{Ccavity}. $$

### Readout Pin Coupling Loss

In [7]:
project.get_quality_factor(design_name='CouplingReadoutPin', modes = [0,1], print_results=True)

INFO 10:17AM [connect]: Connecting to Ansys Desktop API...
INFO 10:17AM [load_ansys_project]: 	File path to HFSS project found.
INFO 10:17AM [load_ansys_project]: 	Opened Ansys App
INFO 10:17AM [load_ansys_project]: 	Opened Ansys Desktop v2020.2.0
INFO 10:17AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/barkayg/Dropbox (Weizmann Institute)/Quantum Circuits Lab/Barkay/SRF Cavity/Simulations/LossesCalculations/LossesCalculationsExample/
	Project:   CavityLosses
INFO 10:17AM [connect]: 	Opened active design
	Design:    CouplingReadoutPin [Solution type: Eigenmode]
INFO 10:17AM [get_setup]: 	Opened setup `Setup1`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 10:17AM [connect]: 	Connection to Ansys established successfully. 😀 

INFO 10:17AM [analyze]: Analyzing setup Setup1
Design "CouplingReadoutPin" info:
	# eigenmodes    7
	# variations    1
      Freq. (GHz)  Quality Factor  Lifetime (s)
mode                                           
0        4.371596    3.548972e+10  

### Drive Pin Coupling Loss

In [8]:
project.get_quality_factor(design_name='CouplingDrivePin', modes = [0,1], print_results=True)

INFO 10:17AM [connect]: Connecting to Ansys Desktop API...
INFO 10:17AM [load_ansys_project]: 	File path to HFSS project found.
INFO 10:17AM [load_ansys_project]: 	Opened Ansys App
INFO 10:17AM [load_ansys_project]: 	Opened Ansys Desktop v2020.2.0
INFO 10:17AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/barkayg/Dropbox (Weizmann Institute)/Quantum Circuits Lab/Barkay/SRF Cavity/Simulations/LossesCalculations/LossesCalculationsExample/
	Project:   CavityLosses
INFO 10:17AM [connect]: 	Opened active design
	Design:    CouplingDrivePin [Solution type: Eigenmode]
INFO 10:17AM [get_setup]: 	Opened setup `Setup1`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 10:17AM [connect]: 	Connection to Ansys established successfully. 😀 

INFO 10:17AM [analyze]: Analyzing setup Setup1
Design "CouplingDrivePin" info:
	# eigenmodes    7
	# variations    1
      Freq. (GHz)  Quality Factor  Lifetime (s)
mode                                           
0        4.371605    6.293457e+11     2

### Cavity Pin Coupling Loss

In [9]:
project.get_quality_factor(design_name='CouplingCavityPin', modes = [0,1], print_results=True)

INFO 10:17AM [connect]: Connecting to Ansys Desktop API...
INFO 10:17AM [load_ansys_project]: 	File path to HFSS project found.
INFO 10:17AM [load_ansys_project]: 	Opened Ansys App
INFO 10:17AM [load_ansys_project]: 	Opened Ansys Desktop v2020.2.0
INFO 10:17AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/barkayg/Dropbox (Weizmann Institute)/Quantum Circuits Lab/Barkay/SRF Cavity/Simulations/LossesCalculations/LossesCalculationsExample/
	Project:   CavityLosses
INFO 10:17AM [connect]: 	Opened active design
	Design:    CouplingCavityPin [Solution type: Eigenmode]
INFO 10:17AM [get_setup]: 	Opened setup `Setup1`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 10:17AM [connect]: 	Connection to Ansys established successfully. 😀 

INFO 10:17AM [analyze]: Analyzing setup Setup1
Design "CouplingCavityPin" info:
	# eigenmodes    7
	# variations    1
      Freq. (GHz)  Quality Factor  Lifetime (s)
mode                                           
0        4.371597    6.456899e+09    

In [10]:
import pandas as pd
df = pd.DataFrame.from_dict(project.quality_factors)
print(df.to_markdown())

|                    |        mode 0 |           mode 1 |
|:-------------------|--------------:|-----------------:|
| seam_loss          |   7.87316e+12 |      4.51939e+07 |
| geometry_factor    | 211.309       |    nan           |
| filling_factor     |   1.30487e-08 |    nan           |
| ResistiveTransmon  |   1.98201e+10 | 677093           |
| SapphireBulkLoss   |   2.29594e+10 |      3.02621e+06 |
| CouplingReadoutPin |   3.54897e+10 |      7.33509e+06 |
| CouplingDrivePin   |   6.29346e+11 |      2.23004e+08 |
| CouplingCavityPin  |   6.4569e+09  |      2.44921e+12 |
